# LINFA Tutorial
This LINFA tutorial will guide through the definition of each of the quantities and functions of LINFA by applying LINFA to a practical problem set.
<br>

* **What is LINFA?**
<br> 
    LINFA is a library for variational inference with normalizing flow and adaptive annealing. LINFA accommodates computationally expensive models and difficult-to-sample posterior distributions with dependent parameters.
* **Why use LINFA?**
<br>
    Designed as a general inference engine, LINFA allows the user to define custom input transformations, computational models, surrogates, and likelihood functions which will be discussed throughout the tutorial.

### Tutorial outline
In this tutorial we will:
1. Define the model (problem set) to apply functions and quantities supported by LINFA.
2. Check if the model gradients computed by PyTorch match with simple finite difference-based approximations.
3. Model evaluation set up process and applications:
    * Application 1: Variational inference with the original model.
    * Application 2: Variational inference with neural network surrogate model.

After going through this tutorial, users should be able to define and integrate their model with LINFA, and use the various features provided by LINFA to perform variational inference.
<br>

In addition, we emphasize two special features available through LINFA:
* Adaptively trained surrogate models (NoFAS module).
* Adaptive annealing schedulers (AdaAnn module).

We encourage the user to take advantage of such modules, especially when using physics-based models with computationally expensive evaluations. It relieves the need of heavy computations such as gradient calculation directly through the model which reduces computational cost of inference, particularly for difficult-to-sample distributions.

### Additional Resources

#### Background theory and examples for LINFA
* Y. Wang, F. Liu and D.E. Schiavazzi, Variational Inference with NoFAS: Normalizing Flow with Adaptive Surrogate for Computationally Expensive Models: https://www.sciencedirect.com/science/article/abs/pii/S0021999122005162
* E.R. Cobian, J.D. Hauenstein, F. Liu and D.E. Schiavazzi, AdaAnn: Adaptive Annealing Scheduler for Probability Density Approximation:
https://www.dl.begellhouse.com/journals/52034eb04b657aea,796f39cb1acf1296,6f85fe1149ff41d9.html?sgstd=1


#### More about LINFA library: 
* LINFA library [documentation](https://linfa-vi.readthedocs.io/en/latest/index.html).
* LINFA GitHub [repository](https://github.com/desResLab/LINFA)

In [ ]:
## Import libraries ##
import os
from linfa.run_experiment import experiment
from linfa.transform import Transformation
from linfa.nofas import Surrogate
import torch
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

### Problem definition
* Our physics-based model **phys** consists of a simple ballistic model. We would like to compute the quantities: 
    * maximum height (m) $x_{1}$,
    * final landing location (m) of the object $x_{2}$,
    * total flight time (s)  $x_{2}$,
    
    from the inputs:
    * starting position (m) $z_{1}$,
    * initial velocity (m/s)  $z_{2}$,
    * angle (degrees) $z_{3}$.
<br>

The model is described by the following equations
$$
x_{1} = \frac{z_{2}^{2}\,\sin^{2}(z_{3})}{2\,g},\,\,
x_{2} = z_{1} + \frac{z_{2}^{2}\,\sin(2\,z_{3})}{g},\,\,
x_{3} = \frac{2\,z_{2}\,\sin(z_{3})}{g}.
$$

#### Model identifiability

By considering fixed values for the outputs $(x_{1},x_{2},x_{3}) = (\widetilde{x}_{1},\widetilde{x}_{2},\widetilde{x}_{3})$, we can perform some algebraic manipulation. For example if we derive $z_{2}$ from the equation for $x_{3}$ and we plug it back in the equation for $x_{1}$, we get the equation
$$
g\,\frac{\widetilde{x}_{3}^{2}}{8} = \widetilde{x}_{1}^{2}.
$$

This suggests the maximum height and time of flight are, as expected, related by a deterministic condition and therefore only one of these provide an independent information for the solution of the inverse problem. 

Due to this relation, the number of observables is reduced to only two, from three inputs. This results in a non-identifiable inference task. In other words, there is an infinite number of input combinations $(z_{1},z_{2},z_{3})$ corresponding to the outputs $(\widetilde{x}_{1},\widetilde{x}_{2},\widetilde{x}_{3})$. 

A graphical explanation for this lack of identifiability can be is shown in the the plot below

<img src="./imgs/trajectories.png" width="800px" aligned="center"><br>
Examples of trajectories resulting in the same landing distance and maximum height (or time of flight).

This picture shows how the final target location at $x_{2}$ can be reached by multiple initial positions, velocities and angles. The lack of indetifiability also translates in the existence of a one-dimensional manifold of inputs that correspond to the same outputs. This manifold can be determined from the following expressions for the relations $z_1(z_{3})$ and $z_2(z_{3})$ 
$$
z_{1} = \widetilde{x}_{2} - \frac{g\cdot \widetilde{x}_{3}^{2}}{2}\cdot \left[\frac{\cos(z_{3})}{\sin(z_{3})}\right],\,\,
z_{2} = \frac{g\cdot \widetilde{x}_{3}}{2\,\sin(z_{3})}.
$$
These two curves are also plotted below.

<img src="./imgs/non_ident.png" width="800px" aligned="center"><br>
Two-dimensional projections of one-dimensional manifold where all parameters correspond to the same outputs.

#### Implementation as a Python class

* We now create a new **Phys** model class, having three member functions:
    * `__init__`: A constructor.
    * `genDataFile`: A member function to create synthetic observations.
    * `solve_t`: A function to perform forward model evaluations.
    * *Please refer to the comments below for additional implementation details.* 

In [ ]:
#### Implementation of the traditional trajectory motion physics problem ####
class Phys:
    
    ### Define constructor function for Phys class ###
    def __init__(self):
        ## Define input parameters (True value)  
        # input[] = [starting_position, initial_velocity, angle] = [1(m), 5(m/s), 60(degs)]
        self.defParam = torch.Tensor([[1.0, 5.0, 60.0]])

        self.gConst = 9.81   # gravitational constant
        self.stdRatio = 0.05 # standard deviation ratio
        self.data = None     # data set of model sample

    ### Define data file generator function ###
    # dataSize (int): size of sample (data)
    # dataFileName (String): name of the sample data file
    # store (Boolean): True if user wish to store the generated data file; False otherwise.
    def genDataFile(self, dataSize = 50, dataFileName="data_phys.txt", store=True):
        def_out = self.solve_t(self.defParam)[0]
        print(def_out)
        self.data = def_out + self.stdRatio * torch.abs(def_out) * torch.normal(0, 1, size=(dataSize, 3))
        self.data = self.data.t().detach().numpy()
        if store: np.savetxt(dataFileName, self.data)
        return self.data

    ### Define data file generator function ###
    # params (Tensor): input parameters storing starting position, initial velocity, and angle in corresponding order.
    def solve_t(self, params):
        z1, z2, z3 = torch.chunk(params, chunks=3, dim=1) # input parameters
        z3 = z3 * (np.pi / 180)                           # convert unit from degree to radians
        
        ## Output value calculation
        # ouput[] = [maximum_height, final_location, total_time]
        x = torch.cat(( (z2 * z2 * torch.sin(z3) * torch.sin(z3)) / (2.0 * self.gConst),  # x1: maxHeight
            z1 + ((z2 * z2 * torch.sin(2.0 * z3)) / self.gConst),                         # x2: finalLocation 
            (2.0 * z2 * torch.sin(z3)) / self.gConst), 1)                                 # x3: totalTime
        return x

In [ ]:
## Generate phys sample file ##

# Define model
model = Phys()

# Generate Data
physData = model.genDataFile()

Now that we have our model set up, we go on to our second step, i.e., *Calculating the gradient to confirm model functionality.*

### Check for Gradient Calculation
* Prior to applying NOFAS to our Phys model, we check if the model gradient (Jacobian actually since it has multiple outputs) is correctly computed by PyTorch. 
* Specifically, when the surrogate is not enabled, gradient calculation is completed straight through the model, so we want to ensure that this is correct before running some inference task.
* Here we compute each gradient using (1) Pytorch and (2) a finite difference (Euler forward differences) approximation, and compare the results provided by these two approaches.

 #### Computing gradients through PyTorch

In [ ]:
#### Implementation of gradient calculation using PyTorch - version 2 #### 
class PytorchGrad2: 
    ### Define constructor function for PytorchGrad2 class ###
    def __init__(self, model, transform):
        # Define input parameters and enable gradient calculation
        self.z = torch.Tensor([[1.0, 5.0, 60.0]])
        self.z.requires_grad = True
        
        self.in_vals = transform.forward(self.z)

        self.out_val = model.solve_t(self.in_vals)
        self.out1, self.out2, self.out3 = torch.chunk(self.out_val, chunks=3, dim=1)

    # Compute gradients using backward function for y
    def back_x1(self): 
        self.out1.backward()
        d1 = self.in_vals.grad
        a, b, c = torch.chunk(d1, chunks=3, dim=1)
        return [a, b, c]
    
    def back_x2(self): 
        self.out2.backward()
        d2 = self.in_vals.grad
        a, b, c = torch.chunk(d2, chunks=3, dim=1)
        return [a, b, c]
    
    def back_x3(self): 
        self.out3.backward()
        d3 = self.in_vals.grad
        a, b, c = torch.chunk(d3, chunks=3, dim=1)
        return [a, b, c]
        

In [ ]:
# Define Phys model
model = Phys()
# Set transformation information and define transforamtion
trsf_info = [['identity',0.0,0.0,0.0,0.0],
             ['identity',0,0.0,0.0,0.0],
             ['identity',0,0.0,0.0,0.0]]
        
transform = Transformation(trsf_info)

# List to store dx/dz values
dx_dz_pytorch2 = []

# Define PytorchGrad object and calculate gradient
pyGrad2 = PytorchGrad2(model, transform)
dx_dz_pytorch2.append(pyGrad2.back_x1())

pyGrad2 = PytorchGrad2(model, transform)
dx_dz_pytorch2.append(pyGrad2.back_x2())

pyGrad2 = PytorchGrad2(model, transform)
dx_dz_pytorch2.append(pyGrad2.back_x3())

# print(dx_dz_pytorch2) # check if output matches expectations

# convert to pandas DataFrame for readability
jacob_mat_2 = pd.DataFrame(dx_dz_pytorch2.numpy(), columns=['dz1', 'dz2', 'dz3'])
jacob_mat_2.index = ['dx1', 'dx2', 'dx3']
jacob_mat_2

#### Approximating gradients with finite differences

In [ ]:
### Function that manually calculates a derivative ###
def getGrad(f_eps, f, eps):
    return (f_eps - f) / (eps)

### Function that returns a list of gradients ###
def gradList(f_eps1, f_eps2, f_eps3, f, eps): 
    return [getGrad(f_eps1, f, eps), getGrad(f_eps2, f, eps), getGrad(f_eps3, f, eps)]

In [ ]:
# List to store dx/dz values
dx_dz = []
dx1_dz = []
dx2_dz = []
dx3_dz = []

# Set up parameters
eps = 1.0
z = torch.Tensor([[1.0, 5.0, 60.0]])
z_eps1 = torch.Tensor([[1.0 + eps, 5.0, 60.0]])
z_eps2 = torch.Tensor([[1.0, 5.0 + eps, 60.0]])
z_eps3 = torch.Tensor([[1.0, 5.0, 60.0 + eps]])

x1_eps1 = model.solve_t(z_eps1)[0,0]
x1_eps2 = model.solve_t(z_eps2)[0,0]
x1_eps3 = model.solve_t(z_eps3)[0,0]
x1_eps = model.solve_t(z)[0,0]

dx1_dz = gradList(x1_eps1, x1_eps2, x1_eps3, x1_eps, eps)
dx_dz.append(dx1_dz)

x2_eps1 = model.solve_t(z_eps1)[0,1]
x2_eps2 = model.solve_t(z_eps2)[0,1]
x2_eps3 = model.solve_t(z_eps3)[0,1]
x2_eps = model.solve_t(z)[0,1]

dx2_dz = gradList(x2_eps1, x2_eps2, x2_eps3, x2_eps, eps)
dx_dz.append(dx2_dz)

x3_eps1 = model.solve_t(z_eps1)[0,2]
x3_eps2 = model.solve_t(z_eps2)[0,2]
x3_eps3 = model.solve_t(z_eps3)[0,2]
x3_eps = model.solve_t(z)[0,2]

dx3_dz = gradList(x3_eps1, x3_eps2, x3_eps3, x3_eps, eps)
dx_dz.append(dx3_dz)

# print(dx_dz) # check if values match expected outputs

# convert to pandas DataFrame for readability
jacob_mat_3 = pd.DataFrame(dx_dz, columns=['dz1', 'dz2', 'dz3'])
jacob_mat_3.index = ['dx1', 'dx2', 'dx3']
jacob_mat_3

#### We verify the convergence for the finite difference approximation to the PyTorch gradient for dx2_dz3
- Note: adjust values to check convergence for other gradients of interest

In [ ]:
## Focus: dx2_dz3

initial_eps = 15        # Initial change of value (eps)
k = 150                 # Number of iterations
dx2_dz3_list = []       # List to store results
pytorch_grad2 = -0.0445 # Pytorch gradient value

# Calculate for dx2_dz3 as eps decreases
for t in range(1, k):
    update_eps = initial_eps*(1/t)                             # updated eps value
    z_eps3 = torch.Tensor([[1.0, 5.0, 60.0 + update_eps]])     # update z_eps3
    x2_eps3 = model.solve_t(z_eps3)[0,1]                       # update x2_eps3
    dx2_dz3_list.append(getGrad(x2_eps3, x2_eps, update_eps))  # store result to dx2_dz3_list

In [ ]:
## Plot result to see convergence

fig, ax = plt.subplots()
ax.plot(range(1,k), dx2_dz3_list, c = "red", linestyle = "solid", label = "Model Gradient")

plt.axhline(y = pytorch_grad2, color = 'blue', linestyle = '-', label = "Pytorch Gradient-2")
plt.legend(loc="lower right")
plt.title("Gradient Plot for dx2_dz3")
plt.ylabel("Gradient")
plt.xlabel("k-Iterations")
plt.show()

Now that we confirmed that our model successfully computes the gradients, we go on to our third step: *Model Evaluation Set Up and Applications*

### Variational inference with full model

#### Definition of hyperparameters
The first step is to define all options and hyperparameters for the inference task. Additional detail for each hyperparameter can be found in the [documentation](https://linfa-vi.readthedocs.io/en/latest/content/linfa_options.html) or in the definition of the [experiment](https://github.com/desResLab/LINFA/blob/master/linfa/run_experiment.py) class.

In [ ]:
# Experiment Setting
exp = experiment()
exp.flow_type        = 'maf'        # str: Type of flow
exp.n_blocks         = 5            # int: Number of layers                            
exp.hidden_size      = 100          # int: Hidden layer size for MADE in each layer  
exp.n_hidden         = 1            # int: Number of hidden layers in each MADE      
exp.activation_fn    = 'relu'       # str: Actication function used                  
exp.input_order      = 'sequential' # str: Input order for create_mask           
exp.batch_norm_order = True         # boolean: Order to decide if batch_norm is used    
exp.save_interval    = 5000         # int: How often to sample from normalizing flow

exp.input_size    = 3               # int: Dimensionality of input                   
exp.batch_size    = 250             # int: Number of samples generated             
exp.true_data_num = 2               # double: number of true model evaluated      
exp.n_iter        = 25001           # int: Number of iterations                      
exp.lr            = 0.01            # float: Learning rate                              
exp.lr_decay      = 0.9999          # float: Learning rate decay                
exp.log_interval  = 100             # int: How often to show loss stat   

exp.run_nofas          = False      # boolean: to run experiment with nofas
exp.annealing          = False      # boolean: to run experiment with annealing
exp.calibrate_interval = 1000       # int: How often to update surrogate model     
exp.budget             = 260        # int: Total number of true model evaluation

exp.surr_pre_it  = 20000            # int: Number of pre-training iterations for surrogate model
exp.surr_upd_it  = 6000             # int: Number of iterations for the surrogate model update
exp.surr_folder  = "./"
exp.use_new_surr = True             # boolean: to run experiment with nofas

exp.results_file = 'results.txt'      # str: result text file name
exp.log_file     = 'log.txt'          # str: log text file name
exp.samples_file = 'samples.txt'      # str: sample text file name
exp.seed         = random.randint(0, 10 ** 9)  # int: Random seed used
exp.n_sample     = 5000               # int: Total number of iterations
exp.no_cuda      = True               # boolean: to run experiment with NO cuda

exp.optimizer    = 'RMSprop'          # str: Type of optimizer
exp.lr_scheduler = 'ExponentialLR'    # str: Type of scheduler

exp.device = torch.device('cuda:0' if torch.cuda.is_available() and not exp.no_cuda else 'cpu')

#### Define the transformation 
Now we define the trasformation of parameters and initialize the 

In [ ]:
# Define transformation based on normalization rate
trsf_info = [['identity',0.0,0.0,0.0,0.0],
             ['identity',0.0,0.0,0.0,0.0],
             ['linear',-3,3,30.0,80.0]]
trsf = Transformation(trsf_info)        
exp.transform = trsf

#### Model and surrogate definition
We create an instance of the **Phys** model and assign `None` to the surrogate.

In [ ]:
# Define model
model = Phys()
exp.model = model

# Get data
model.data = np.loadtxt('./data_phys.txt')

# Run experiment without surrogate
exp.surrogate = None

#### Log-likelihood definiton


In [ ]:
## Define log density
# x: original, untransformed inputs
# model: our model
# transform: our transformation 
def log_density(x, model, surrogate, transform):
    # x contains the original, untransformed inputs
    np.savetxt(exp.output_dir + '/' + exp.name + '_x', x.detach().numpy(), newline="\n")
    # Compute transformation log Jacobian
    adjust = transform.compute_log_jacob_func(x)

    batch_size = x.size(0)
    # Get the absolute values of the standard deviations
    stds = torch.abs(model.solve_t(model.defParam)) * model.stdRatio
    Data = torch.tensor(model.data).to(exp.device)
    
    # Check for surrogate
    if surrogate:
        modelOut = exp.surrogate.forward(x)
    else:
        modelOut = model.solve_t(transform.forward(x))

    # Eval LL
    ll1 = -0.5 * np.prod(model.data.shape) * np.log(2.0 * np.pi)
    ll2 = (-0.5 * model.data.shape[1] * torch.log(torch.prod(stds))).item()
    ll3 = 0.0
    for i in range(3):
        ll3 += - 0.5 * torch.sum(((modelOut[:, i].unsqueeze(1) - Data[i, :].unsqueeze(0)) / stds[0, i]) ** 2, dim=1)
    negLL = -(ll1 + ll2 + ll3)
    res = -negLL.reshape(x.size(0), 1) + adjust
    np.savetxt(exp.output_dir + '/' + exp.name + '_res', res.detach().numpy(), newline="\n")
    return res

#### Launch inference task

In [ ]:
## Run 
print('')
print('--- Temporary TEST: Physics Example - without NOFAS')
print('')

print('--- Running on device: '+ str(exp.device))
print('')

# Experiment Setting
exp.name = "phys_nofasFree"           # str: Name of experiment
exp.output_dir   = './' + exp.name    # str: output directory location

# Assign logdensity
exp.model_logdensity = lambda x: log_density(x, model, exp.surrogate, trsf)

# Run VI
exp.run()

Notice that the model evaluation has been successfully completed by checking at the newly created **phys_nofasFree** folder in our current directory.
Note also that LINFA supports a post processing script to plot all results which includes plots of log loss, parameter estimation, and estimated output data.

The according code line is: `python -m linfa.plot_res -n phys_nofasFree -i 25000 -f phys_nofasFree`
    
> However, even with our simple model, computing the gradients and confirming model functionality everytime before applying the model evaluation is time consuming and when it comes with evaluting even more complex models, the computational costs will be exponential and will likely result in intractable inference. <br>
In such cases, LINFA enables the construction of the adaptively trained surrogate model which resolves such concerns regarding the computational cost of inference. By utilizing the surrogate model, gradient computation is executed by the surrogate model which eliminates the need to manually check for gradient calcultation. <br>
In addition, LINFA provides an adaptive annealing scheduler which allows easier sampling from complicated densities. <br>

> Accordingly, we will specifically observe how the adaptively trained surrogate model relieves compuatational cost while ensuring inference in our last step: <br>
*Applying our model including the Surrogate model.*

#### Results
The results below are obtained from variational inference with the full model. The posterior distribution is concentrated around the one-dimensional manifold where the input parameters are not identifiable. 

<img src="./imgs/orig_log.png" width="300px" aligned="center"><br>
Loss converge profiles for variational inference with the full model.<br>
<br>

<img src="./imgs/orig_params_1.png" width="300px" aligned="center">
<img src="./imgs/orig_params_2.png" width="300px" aligned="center">
<img src="./imgs/orig_params_3.png" width="300px" aligned="center"><br>
Posterior distribution of the input parameters.<br>
<br>

<img src="./imgs/orig_data_1.png" width="300px" aligned="center">
<img src="./imgs/orig_data_2.png" width="300px" aligned="center">
<img src="./imgs/orig_data_3.png" width="300px" aligned="center"><br>
Comparison between posterior predictive distribution and available observations.<br>
<br>

In [ ]:
from IPython.display import IFrame
IFrame("./samples/simple3.pdf", width=600, height=300)

### 4. Phys Model with the Adaptively Trained Surrogate Model
Note: this block of code takes a while.

In [ ]:
# Experiment Setting
exp.name = "phys"      # str: Name of experiment
exp.output_dir   = './' + exp.name    # str: output directory location

# Define model
model = Phys()
exp.model = model

# Get data
model.data = np.loadtxt('./data_phys.txt')

# Define surrogate
exp.surrogate = Surrogate(exp.name, lambda x: model.solve_t(trsf.forward(x)), exp.input_size, 3, 
                          model_folder=exp.surr_folder, limits=torch.Tensor([[0, 2], [0, 10], [-3, 3]]), 
                          memory_len=20, device=exp.device)
surr_filename = exp.surr_folder + exp.name
if exp.use_new_surr or (not os.path.isfile(surr_filename + ".sur")) or (not os.path.isfile(surr_filename + ".npz")):
    print("Warning: Surrogate model files: {0}.npz and {0}.npz could not be found. ".format(surr_filename))
    # 4 samples for each dimension: pre-grid size = 16
#     exp.surrogate.gen_grid(gridnum=4)
    exp.surrogate.gen_grid(gridnum=3)
    exp.surrogate.pre_train(exp.surr_pre_it, 0.03, 0.9999, 500, store=True)
# Load the surrogate
exp.surrogate.surrogate_load()

In [ ]:
## Run 
print('')
print('--- Temporary TEST: Physics Example - with NOFAS')
print('')

print('--- Running on device: '+ str(exp.device))
print('')

# Assign logdensity
exp.model_logdensity = lambda x: log_density(x, model, exp.surrogate, trsf)

# Run VI
exp.run()

> Notice that the model evaluation has been successfully completed by checking at the newly created **phys** folder in our current directory.<br>
> Note that LINFA supports a post processing script to plot all results which includes plots of log loss, parameter estimation, and estimated output data. <br>
The according code line is: `python -m linfa.plot_res -n phys -i 25000 -f phys`
<br>

    
> Now, to compare the functionalities of the surrogate model, we observe the plots generated by the LINFA library.
<br>
Note that the generated plots below are converted to png format.

In [ ]:
# import libraries
import cv2
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
from IPython.display import IFrame

## Phys. without Surrogate
# 1-a) output evaluation plots - without surrogate
output1_free = convert_from_path('./phys_nofasFree/data_plot_phys_nofasFree_25000_0_1.pdf')
for page in output1_free:
    page.save('data_plot_phys_nofasFree_25000_0_1.png', 'PNG')
    
output2_free = convert_from_path('./phys_nofasFree/data_plot_phys_nofasFree_25000_0_2.pdf')
for page in output2_free:
    page.save('data_plot_phys_nofasFree_25000_0_2.png', 'PNG')
    
output3_free = convert_from_path('./phys_nofasFree/data_plot_phys_nofasFree_25000_1_2.pdf')
for page in output3_free:
    page.save('data_plot_phys_nofasFree_25000_1_2.png', 'PNG')
    
# 1-b) output evaluation plots - with surrogate
output1 = convert_from_path('./phys/data_plot_phys_25000_0_1.pdf')
for page in output1:
    page.save('data_plot_phys_25000_0_1.png', 'PNG')
    
output2 = convert_from_path('./phys/data_plot_phys_25000_0_2.pdf')
for page in output2:
    page.save('data_plot_phys_25000_0_2.png', 'PNG')
    
output3 = convert_from_path('./phys/data_plot_phys_25000_1_2.pdf')
for page in output3:
    page.save('data_plot_phys_25000_1_2.png', 'PNG')
    
# 2-a) log-loss plots - without surrogate
log_free = convert_from_path('./phys_nofasFree/log_plot.pdf')
for page in log_free:
    page.save('log_plot_nofasFree.png', 'PNG')

# 2-b) log-loss plots - with surrogate
log_free = convert_from_path('./phys/log_plot.pdf')
for page in log_free:
    page.save('log_plot_phys.png', 'PNG')
    
# 3-a) parameter evaluation plots - without surrogate
param1_free = convert_from_path('./phys_nofasFree/params_plot_phys_nofasFree_25000_0_1.pdf')
for page in param1_free:
    page.save('params_plot_phys_nofasFree_25000_0_1.png', 'PNG')  
    
param2_free = convert_from_path('./phys_nofasFree/params_plot_phys_nofasFree_25000_0_2.pdf')
for page in param2_free:
    page.save('params_plot_phys_nofasFree_25000_0_2.png', 'PNG')  
    
param3_free = convert_from_path('./phys_nofasFree/params_plot_phys_nofasFree_25000_1_2.pdf')
for page in param3_free:
    page.save('params_plot_phys_nofasFree_25000_1_2.png', 'PNG')  
    
# 3-b) parameter evaluation plots - with surrogate
param1 = convert_from_path('./phys/params_plot_phys_25000_0_1.pdf')
for page in param1:
    page.save('params_plot_phys_25000_0_1.png', 'PNG')    
    
param2 = convert_from_path('./phys/params_plot_phys_25000_0_2.pdf')
for page in param2:
    page.save('params_plot_phys_25000_0_2.png', 'PNG')   
    
param3 = convert_from_path('./phys/params_plot_phys_25000_1_2.pdf')
for page in param3:
    page.save('params_plot_phys_25000_1_2.png', 'PNG')      

In [ ]:
## We first take a look to check if the images are well converted to png

# import package
from IPython.display import Image, display

# display png image
img1 = Image(filename='output1_free.png')
img2 = Image(filename='output2_free.png')
img3 = Image(filename='output3_free.png')
# display(img1, img2, img3)

> Now we compare the generated plots of log-loss, output data and parameter estimation.

### 1) *log-loss* plots

In [ ]:
## log-loss plots

# create figure
fig = plt.figure(figsize=(10, 7))

# setting row and column variables
rows = 1
columns = 2

# reading images
logLoss_phys = cv2.imread('log_plot_phys.png')
logLoss_physFree = cv2.imread('log_plot_nofasFree.png')

# Adds a subplot at the 1st position
fig.add_subplot(rows, columns, 1)

# Add subplots corresponding to Phys with Surrogate
plt.imshow(logLoss_phys)
plt.axis('off')
plt.title("Phys. with Surrogate")

# Add subplots corresponding to Phys without Surrogate
fig.add_subplot(rows, columns, 2)
# showing image
plt.imshow(logLoss_physFree)
plt.axis('off')
plt.title("Phys. without Surrogate")


> While both of the log loss plots show convergence to a lower log loss value, notice that the log loss plot of the Phys. model that utilized the Surrogate model had a lower log loss value than the case when the Surrogate was not used.
<br>

### 2) *ouput estimation* plots

In [ ]:
## output plots

# create figure
fig = plt.figure(figsize=(10, 7))

# setting row and column variables
rows = 3
columns = 2

# reading images
phys1 = cv2.imread('data_plot_phys_25000_0_1.png')
phys2 = cv2.imread('data_plot_phys_25000_0_2.png')
phys3 = cv2.imread('data_plot_phys_25000_1_2.png')
# print('Image Width is',Image1.shape[1]) #327
# print('Image Height is',Image1.shape[0]) #259
# Image1 = cv2.resize(Image1, (400,300))

physFree1 = cv2.imread('data_plot_phys_nofasFree_25000_0_1.png')
physFree2 = cv2.imread('data_plot_phys_nofasFree_25000_0_2.png')
physFree3 = cv2.imread('data_plot_phys_nofasFree_25000_1_2.png')

# Add subplots corresponding to Phys with Surrogates
fig.add_subplot(rows, columns, 1)
plt.imshow(phys1)
plt.axis('off')
plt.title("Phys. with Surrogate")
fig.add_subplot(rows, columns, 3)
plt.imshow(phys2)
plt.axis('off')
fig.add_subplot(rows, columns, 5)
plt.imshow(phys3)
plt.axis('off')

# Add subplots corresponding to Phys without Surrogate
fig.add_subplot(rows, columns, 2)
plt.imshow(physFree1)
plt.axis('off')
plt.title("Phys. with Surrogate")
fig.add_subplot(rows, columns, 4)
plt.imshow(physFree2)
plt.axis('off')
fig.add_subplot(rows, columns, 6)
plt.imshow(physFree3)
plt.axis('off')

> While both of the output plots have meaningful results where the samples (blue dots) are distributed within the estimated region (red cluster), notice that the samples are much more clustered within the estimated area from the Phys. model that utilized the Surrogate model than the case where the Surrogate model was not used.
<br>

### 3) *parameter estimation* plots

In [ ]:
## parameter estimation plots

# create figure
fig = plt.figure(figsize=(10, 7))

# setting row and column variables
rows = 3
columns = 2

# reading images
phys1 = cv2.imread('params_plot_phys_25000_0_1.png')
phys2 = cv2.imread('params_plot_phys_25000_0_2.png')
phys3 = cv2.imread('params_plot_phys_25000_1_2.png')

physFree1 = cv2.imread('params_plot_phys_nofasFree_25000_0_1.png')
physFree2 = cv2.imread('params_plot_phys_nofasFree_25000_0_2.png')
physFree3 = cv2.imread('params_plot_phys_nofasFree_25000_1_2.png')

# Add subplots corresponding to Phys with Surrogates
fig.add_subplot(rows, columns, 1)
plt.imshow(phys1)
plt.axis('off')
plt.title("Phys. with Surrogate")
fig.add_subplot(rows, columns, 3)
plt.imshow(phys2)
plt.axis('off')
fig.add_subplot(rows, columns, 5)
plt.imshow(phys3)
plt.axis('off')

# Add subplots corresponding to Phys without Surrogate
fig.add_subplot(rows, columns, 2)
plt.imshow(physFree1)
plt.axis('off')
plt.title("Phys. without Surrogate")
fig.add_subplot(rows, columns, 4)
plt.imshow(physFree2)
plt.axis('off')
fig.add_subplot(rows, columns, 6)
plt.imshow(physFree3)
plt.axis('off')

> Recall our input values (parameters) were [1,5,60]. Notice that both cases successfully returns parameter estimations that are fairly accurate. Specifically, we observe that when the Surrogate model is used, the accuracy of the parameter estimation increases.
<br>

> Based on the generated plots, we've observed the physical benefits of the Surrogate model and the functionality of the Phys. model estimation utilizing LINFA. 
> Moreover, LINFA supports various model types. For more information, please refer to our paper **[need to put our paper url!!!]** *Appendix B. Detailed numerical benchmarks* where examples utilizing the LINFA library is introduced.